In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
import sys
import os
#import importlib
# Añadir la ruta de la carpeta donde está el módulo
#sys.path.append(os.path.abspath('./Modulo'))

In [7]:
'''
Lectura de Credenciales en config.json

'''

import json

with open('config.json', 'r') as configfile:
    config = json.load(configfile)
#print(config["sql_server"]["servidor"])


In [10]:
import Modulo.DBConnection as db
from Modulo.StructureMigration import Migration

#Datos de Entrada
O_database = 'AdventureWorksDW2017'
O_schema ='dbo'
O_table = 'DimEmployee'

D_database = 'PRUEBA'
D_schema ='dbo'
D_table = 'DimEmployee'

#Conexión a fuente de datos
OriginCreds = config["sql_server"]
OriginCon = db.SQLServer(OriginCreds["user"], OriginCreds["password"], OriginCreds["server"], O_database, OriginCreds["port"])
OriginResult = OriginCon.crear_conexion()

#Conexión a destino de datos
DestinyCreds = config["sql_server"]
DestinyCon = db.SQLServer(OriginCreds["user"], OriginCreds["password"], OriginCreds["server"], D_database, OriginCreds["port"])
DestinyResult = DestinyCon.crear_conexion()

if OriginResult and DestinyResult:
    print('Conexión Realizada con exito a Origen y Destino!')

    migracion = Migration()
    migracion.setOrigin(OriginCon,O_database, O_schema, O_table)
    migracion.setDestiny(DestinyCon,D_database, D_schema, D_table)
   
    print(f'La migración es la siguiente: {migracion.migrationType()}')

else:
    print('Error en la conexión:\n{}')


Conectando
Conectando
Conexión Realizada con exito a Origen y Destino!
La migración es la siguiente: sql_server --> sql_server


In [20]:
columns = migracion.getOriginStr()
columns.head(5)

,ORDEN,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,COLUMN_NAME,IS_NULLABLE,DATA_TYPE,CHARACTER_MAXIMUM_LENGTH,NUMERIC_PRECISION,NUMERIC_SCALE
0,1,AdventureWorksDW2017,dbo,DimEmployee,EmployeeKey,NO,int,NaN,10.0,0.0
1,2,AdventureWorksDW2017,dbo,DimEmployee,ParentEmployeeKey,YES,int,NaN,10.0,0.0
2,3,AdventureWorksDW2017,dbo,DimEmployee,EmployeeNationalIDAlternateKey,YES,nvarchar,15.0,NaN,NaN
3,4,AdventureWorksDW2017,dbo,DimEmployee,ParentEmployeeNationalIDAlternateKey,YES,nvarchar,15.0,NaN,NaN
4,5,AdventureWorksDW2017,dbo,DimEmployee,SalesTerritoryKey,YES,int,NaN,10.0,0.0


In [17]:
#import Modulo.QueryTemplates as qt
#query = qt.templates["sql_server"]["TableCreation"].replace('@@DATABASE@@', self.OriginDatabase).replace('@@SCHEMA@@', self.OriginSchema).replace('@@TABLE@@', self.OriginTable)
#query = query.replace('@@COLUMNS@@', columns)

columns[["SYNTAX"]] 

"\n\t\tUSE @@DATABASE@@\n\t\tGO\n\n\t\tIF NOT EXISTS (SELECT SCHEMA_ID FROM sys.schemas WHERE [name] = '@@SCHEMA@@')\n\t\tBEGIN\n\t\t\tEXECUTE('CREATE SCHEMA @@SCHEMA@@');\n\t\tEND;\n\t\tGO\n\n\t\tCREATE TABLE @@SCHEMA@@.@@TABLE@@ (\n\t\t@@COLUMNAS@@\n\t\t)\n\t\tGO\n\t\t"